In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import scipy
import sys
import os
import pandas as pd
import mne
import matplotlib
import h5py
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = [17]
#specify which erp we are analyzing
erps = 'lrp'
roi = 'latcen'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access trial type epoched data for each participant
def load_tfr_epochs(pp_num, root_dir, erp_path, task):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'p%03d_%s-epo.fif' % (pp_num, task))

    epochs = mne.read_epochs(filename)

    return epochs, pp_directory

In [3]:
#set up parameters for time-frequency morlet convolution
def set_tfr_params(l_freq = 6, h_freq = 35, num_freq = 50, cycle = 6, samp_rate = 200):
    freqs = np.logspace(*np.log10([l_freq, h_freq]), num = num_freq)
    n_cycles = cycle
    sfreq = samp_rate
    
    return freqs, n_cycles, sfreq

In [4]:
#save tfr'd data
def save_tfr_data(pp_num, root_dir, erp_path, data, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    mne.time_frequency.write_tfrs(out_fname, tfr=data, overwrite = True)

In [5]:
#load tfr'd data
def load_tfr_data(pp_num, root_dir, erp_path, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    out_fname = os.path.join(pp_directory, 'p%03d_%s_%s-tfr.h5' % (pp_num, task, output))
    dat = mne.time_frequency.read_tfrs(out_fname)
    
    return dat

In [6]:
# After transforming the TFR data into a single signal representative of frequency and channels
# we compare the two signals: same as taking their difference (signal1 minus signal2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
    np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0

In [7]:
# save average tfr per participant for each condition
# morlet wavelet convolution for each participant, according to trial types - POWER

perturb_conds = ['smlerrors_rot']

for pcond in range(0, len(perturb_conds)):
    for pp in participants:
        data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
        
        freqs, n_cycles, sfreq = set_tfr_params()
        
        #average epochs within participant
        power = mne.time_frequency.tfr_morlet(
            data,
            freqs=freqs,
            n_cycles=n_cycles,
            use_fft=False,
            decim=1,
            n_jobs=None,
            return_itc = False,
            average = False
        )
        
        save_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, data = power, task = perturb_conds[pcond], output = "trialbytrial_power")

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p017\lrp\p017_smlerrors_rot-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
36 matching events found
No baseline correction applied
0 projection items activated


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s


Not setting metadata


[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   12.3s finished


In [8]:
# load in tfr object, ensure to baseline correct; (tfr shape is channels, freqs, timepts)
# narrow down to channels 
# narrow down to frequencies and timepts
# take mean of theta, alpha, or beta ranges (freqs)
# take mean across electrodes (channels)

perturb_conds = ['smlerrors_rot']

# lateral central CHANNELS
channels = ['C5', 'C3',
            'CP5', 'CP3', 'CP1',
            'P5', 'P3', 'P1']


# BASELINE FOR FEEDBACK ONSET
baseline_t = (-1.3, -1.0)

# BETA
freq_lower = 13
freq_upper = 25


beta_latcen_SmallRot = []


for pcond in range(0, len(perturb_conds)):
    for pp in participants:
        dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond], output = "trialbytrial_power")
        dat = dat[0]
        # apply baseline correction
        dat = dat.apply_baseline(baseline_t, mode='mean')
        # narrow down to channels
        dat = dat.pick_channels(channels)
        #transform to ndarray (channels, freqs, times)
        npdat = dat.data

        # get needed frequency indices
        nfreqs = [] #get indices of frequencies we want
        for i in range(0, len(dat.freqs)):
            if dat.freqs[i] >= freq_lower and dat.freqs[i] <= freq_upper:
                nfreqs.append(i)
        npdat = npdat[:,:, nfreqs, :]

        # get needed timept indices
        ntimes = list(range(200,501))
        npdat = npdat[:,:,:,ntimes]
        
        ppdat = np.mean(npdat, axis = 2) #calculate mean across frequencies
        ppdat = np.mean(ppdat, axis = 1) # calculate mean across channels
        ppdat = ppdat*1e12 #convert V^2 to uV^2
        
        beta_latcen_SmallRot.append(ppdat)
    

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p017/lrp\p017_smlerrors_rot_trialbytrial_power-tfr.h5 ...
Not setting metadata
Applying baseline correction (mode: mean)


In [9]:
# Get timepts to use for indices given by cluster-based permutation later and for timepts in saved data frames
# But only grab -1.5 to 0 sec time-locked to feedback onset (idx= 100:401)
root_directory = root
pp = 0 #only need one participant

# we can use aligned data
dat = load_tfr_data(pp_num = pp, root_dir = root, erp_path = erps, task = 'small_large_aligned', output = "power")
time = dat[0].times
time = time[200:501] #get only timepoints we want

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/p000/lrp\p000_small_large_aligned_power-tfr.h5 ...


In [10]:
# save tfr data across participants in each condition (used for plotting signals)
beta_flists = [beta_latcen_SmallRot]

beta_conditionnames = ['beta_smallrot']

for f in range(0, len(beta_flists)):
    df = pd.DataFrame(beta_flists[f][0]).T
    df['time'] = time

    filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'TFR_trialpower_latcen_%s_%s.csv' % (beta_conditionnames[f], erps))
    df.to_csv(filename)
            

In [ ]:
# Each epoch/ trial can be investigated from the csv file
# quick inspection shows that trial 2 for participant 17, contains extreme power values in this condition
# Remove this single trial